# Outline

## Direct CNOT

The direct CNOT can be obtained using the Hamiltonian from Schrieffer-Wolff perturbation theory. 
\begin{equation}
    H = \Omega_C(t) (g_{IX} IX + g_{ZX} ZX)
\end{equation}
We remark two things: 
1. The control qubit operators commute with Z, while the target qubit operators do not.
2. The relative strength of the coefficients depends on the pair of qubits involved. The unitary trajectory of the gate, and therefore the relative crosstalk contribution, will vary slightly.

### Remark 1

Paying attention to the fact that the control commutes with Z but the target does not, we draw a few additional conclusions. If we are robust to $IZ$ within the gate, then we are in fact robust to $ZZ$. Loosely speaking,
\begin{equation}
    U^\dag(t) Z_C Z_T U(t) = U^\dag(t) I_C Z_T U(t) U^\dag(t) Z_C I_T U(t) = U^\dag(t) I_C Z_T U(t) Z_C I_T
\end{equation}
and only the target term contributes.

The control crosstalk remains unless we use an additional drive to cancel it. We can drive an identity gate on the 

### Remark 2

We can make point 2 clearer if we absorb one of the coefficients into the control drive. Then,
\begin{equation}
    H = \widetilde{\Omega}_C(t) (IX + \mu ZX).
\end{equation}

Of course, we can also drive on the target, so
\begin{equation}
    H = \widetilde{\Omega}_C(t) (IX + \mu ZX) + \Omega_T(t) IX.
\end{equation}
Now we have relative control over the IX rate, so we could optimize for the same unitary dynamics for any gate. The pulse shapes will be different, but the robustness behavior will be what we want.


## Echo CR

Many of the previous insight carry over.

The ZZ crosstalk within the Echo CR vanishes.

The ZZ crosstalk of spectators vanishes. Which?

In [ ]:
using QuantumCollocation
using NamedTrajectories
using TrajectoryIndexingUtils

using CairoMakie
using DelimitedFiles
using Distributions
using LinearAlgebra

In [ ]:
H_crosstalk = (
    kron_from_dict("ZZ", H_operators),
    kron_from_dict("IZ", H_operators),
    kron_from_dict("ZI", H_operators),
)